This is my personal implementation of the Vision Transformer (https://arxiv.org/pdf/2010.11929v2.pdf). 
This approach uses the original Transformer architecture but applies it to images.

I documented alfl of my learning process in commits on the 'ViT_dev' branch, and then just merged the final result into 'main' once I was done. If you'd like
to see my struggles, please feel free to checkout ViT_dev and enjoy! 

Import packages

In [1]:
import math

import torch
import torchvision
from torch import nn